In [1]:
import pandas as pd
import numpy as np
from sglang import function, system, user, assistant, gen, set_default_backend, RuntimeEndpoint
import sglang as sgl
from torch import save


set_default_backend(RuntimeEndpoint("http://localhost:30000"))

@sgl.function
def text_qa(s, question):
    s += question
    s += sgl.gen("answer", stop="]", max_tokens=100,temperature=0)

# Step 1: Read the inputs.csv file and sample 50 entries
df_inputs = pd.read_csv('inputs.csv') # index,date,input
#df_inputs = df_inputs.sample(50)



In [2]:
df_inputs.sample(1)

,index,date,input
3863,3907.0,2011-06-30 13:20:12,User:\nHere is a bitcoin forum thread:\n\n```t...


In [3]:
# Function to split DataFrame into chunks
def split_dataframe(df, chunk_size):
    # Use np.array_split to handle uneven sizes automatically
    chunks = np.array_split(df, len(df) // chunk_size + (1 if len(df) % chunk_size else 0))
    return chunks

# Split the DataFrame into chunks of size 5
# df_chunks = split_dataframe(df_inputs, 2000)
df_chunks = split_dataframe(df_inputs, 200)

for thing in df_chunks:
    print(len(thing))

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199
199


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


In [4]:
for i, df_chunk in enumerate(df_chunks):
    inputs = df_chunk['input'].tolist()
    print(inputs[0]+ """
```json
{
  "categories": [ 
""")
    break

User:
Here is a bitcoin forum thread:

```thread
Date: 2010-01
Topic: Is my second Transaction working correctly? +Transfer Question
### Original post:
So, I finally got my client to start generating and after I got my laptop optimized for the task and waited over night. My first transaction completed resulting in +50 Coins. Yay!!However, once that transaction was complete, a new transaction hasn't started. Or maybe it has. There's only one transaction in the list but I'm up to 131 Blocks under "Status". Is this the way it's supposed to happen? Does it keep processing on the same transaction and generating coins every 120 blocks or so? Or is it supposed to start a new transaction?I have a Screen Shot if needed. Sorry if this is a basic question but I'm sure someone will ask if I don't.Thanks for taking the initiative on this project Mr. Satoshi. Maybe one day I can fire my bank once and for all.EDIT: I decided to stop playing around and I set BitCoin up on my high powered gaming rig, f

In [ ]:
# Process each chunk
for i, df_chunk in enumerate(df_chunks):
    output_loc = f'outputs/outputs_chunk_{i+1}.csv'
    import os
    if os.path.exists(output_loc):
        print(f"Skipping chunk {i+1} as it already exists")
        continue
    inputs = df_chunk['input'].tolist()
    dates = df_chunk['date'].tolist()
    indices = df_chunk['index'].tolist()
    
    # Run batch inference for the current chunk
    states = text_qa.run_batch(
        [{"question": 
("<s>"+input+"""
```json
{
  "categories": [ 
""")
         } for input in inputs],
        progress_bar=True
    )
    
    texts = []
    for state in states:
        for out in state.text_iter():
            texts.append(out.split("Sure! Here is the requested JSON document:")[1].strip())
    
    # Save the outputs of the current chunk to a separate file
    # save(texts, f'outputs_chunk_{i+1}.pt')
    chunk_df = pd.DataFrame({
        'index': indices,
        'date': dates,
        'output': texts
    })
    import os
    os.makedirs('outputs', exist_ok=True)
    chunk_df.to_csv(output_loc, index=False)

Skipping chunk 1 as it already exists
Skipping chunk 2 as it already exists
Skipping chunk 3 as it already exists
Skipping chunk 4 as it already exists
Skipping chunk 5 as it already exists
Skipping chunk 6 as it already exists
Skipping chunk 7 as it already exists
Skipping chunk 8 as it already exists
Skipping chunk 9 as it already exists
Skipping chunk 10 as it already exists
Skipping chunk 11 as it already exists
Skipping chunk 12 as it already exists
Skipping chunk 13 as it already exists
Skipping chunk 14 as it already exists
Skipping chunk 15 as it already exists
Skipping chunk 16 as it already exists
Skipping chunk 17 as it already exists
Skipping chunk 18 as it already exists
Skipping chunk 19 as it already exists
Skipping chunk 20 as it already exists
Skipping chunk 21 as it already exists
Skipping chunk 22 as it already exists
Skipping chunk 23 as it already exists
Skipping chunk 24 as it already exists
Skipping chunk 25 as it already exists
Skipping chunk 26 as it already ex

  0%|          | 0/199 [00:00<?, ?it/s]

In [ ]:
h